In [9]:
# Cell 1: Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")
print(f"✓ TensorFlow version: {tf.__version__}")

✓ All libraries imported successfully
✓ TensorFlow version: 2.19.0


In [10]:
# Cell 2: Load the transformed datasets
print("="*50)
print("LOADING CNN-LSTM READY DATASET")
print("="*50)

# Load the datasets
df_transformed = pd.read_csv('cnn_lstm_ready_dataset.csv')
target_data = pd.read_csv('target_data_for_sequences.csv')

print(f"✓ Transformed data shape: {df_transformed.shape}")
print(f"✓ Target data shape: {target_data.shape}")
print(f"✓ Transformed data columns: {len(df_transformed.columns)} features")
print(f"✓ Target data columns: {len(target_data.columns)} target variables")
print(df_transformed.columns)
# Display first few rows
print(f"\nFirst 3 rows of transformed data:")
print(df_transformed.head(3))

LOADING CNN-LSTM READY DATASET
✓ Transformed data shape: (1458, 65)
✓ Target data shape: (1458, 4)
✓ Transformed data columns: 65 features
✓ Target data columns: 4 target variables
Index(['Year', 'CheckTotal', 'is_zero', 'IsRamadan', 'IsEid', 'IsPreRamadan',
       'IsPostRamadan', 'IsLast10Ramadan', 'IsDSF', 'IsSummerEvent',
       'IsNationalDay', 'IsNewYear', 'IsMarathon', 'IsGITEX', 'IsAirshow',
       'IsFoodFestival', 'IsPreEvent', 'IsPostEvent', 'Month_sin', 'Month_cos',
       'DayOfWeek_sin', 'DayOfWeek_cos', 'Meal_Breakfast', 'Meal_Dinner',
       'Meal_Lunch', 'Event_Dubai-Airshow', 'Event_Dubai-Food-Festival',
       'Event_Dubai-Marathon', 'Event_Dubai-Shopping-Festival',
       'Event_Dubai-Summer-Surprises', 'Event_Eid-Adha', 'Event_Flag-Day',
       'Event_GITEX-Technology-Week', 'Event_New-Year-Celebrations',
       'Event_Normal', 'Event_Post-Dubai-Airshow', 'Event_Post-Dubai-Marathon',
       'Event_Post-Eid-Adha', 'Event_Post-Flag-Day',
       'Event_Post-GITEX-Tech

In [11]:
def create_sequences_for_cnn_lstm(df_transformed, target_data, sequence_length=30, forecast_horizon=7):
    """
    Create sequences for CNN-LSTM training from loaded CSV files
    """
    print("="*50)
    print("CREATING SEQUENCES FOR CNN-LSTM")
    print("="*50)
    
    # Parameters
    SEQ_LENGTH = sequence_length  # Look back 30 days
    FORECAST_HORIZON = forecast_horizon  # Predict next 7 days
    
    # Sort by date to ensure proper sequence order
    df_transformed_sorted = df_transformed.sort_values('Date').reset_index(drop=True)
    target_data_sorted = target_data.sort_values('Date').reset_index(drop=True)
    
    # Pivot target data to wide format
    target_pivot = target_data_sorted.pivot_table(
        index='Date', 
        columns=['RevenueCenterName', 'MealPeriod'], 
        values='CheckTotal', 
        fill_value=0
    ).reset_index()
    
    # Create column names for revenue streams
    target_pivot.columns = ['Date'] + [f"{col[0]}_{col[1]}" for col in target_pivot.columns[1:]]
    
    # Ensure same date range
    common_dates = set(df_transformed_sorted['Date']).intersection(set(target_pivot['Date']))
    df_transformed_sorted = df_transformed_sorted[df_transformed_sorted['Date'].isin(common_dates)].reset_index(drop=True)
    target_pivot = target_pivot[target_pivot['Date'].isin(common_dates)].reset_index(drop=True)
    
    # Remove Date column from features
    feature_columns = [col for col in df_transformed_sorted.columns if col != 'Date']
    features = df_transformed_sorted[feature_columns].values
    
    # Target columns (revenue targets)
    target_columns = [col for col in target_pivot.columns if col != 'Date']
    targets = target_pivot[target_columns].values
    
    print(f"✓ Feature shape: {features.shape}")
    print(f"✓ Target shape: {targets.shape}")
    print(f"✓ Number of feature columns: {len(feature_columns)}")
    print(f"✓ Number of target columns: {len(target_columns)}")
    print(f"✓ Target columns: {target_columns}")
    print(f"✓ Sequence length: {SEQ_LENGTH} days")
    print(f"✓ Forecast horizon: {FORECAST_HORIZON} days")
    
    # Create sequences
    X, y = [], []
    
    for i in range(SEQ_LENGTH, len(features) - FORECAST_HORIZON + 1):
        # Features: past 30 days
        X.append(features[i-SEQ_LENGTH:i])
        
        # Targets: next 7 days
        y.append(targets[i:i+FORECAST_HORIZON])
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"✓ Final X shape: {X.shape}")  # (samples, 30, features)
    print(f"✓ Final y shape: {y.shape}")  # (samples, 7, revenue_targets)
    print(f"✓ Total sequences created: {len(X)}")
    
    return X, y, feature_columns, target_columns

In [12]:
# Cell 3A: Target Normalization Functions
from sklearn.preprocessing import StandardScaler
import joblib

def normalize_targets(y_train, y_test, save_scaler=True):
    """
    Normalize target values for better training stability
    """
    print("="*50)
    print("NORMALIZING TARGET VALUES")
    print("="*50)
    
    # Original data info
    print(f"📊 Original target ranges:")
    print(f"  y_train: ${y_train.min():.2f} - ${y_train.max():.2f}")
    print(f"  y_test: ${y_test.min():.2f} - ${y_test.max():.2f}")
    
    # Reshape for normalization: (samples, days, streams) -> (samples*days, streams)
    original_train_shape = y_train.shape
    original_test_shape = y_test.shape
    
    y_train_reshaped = y_train.reshape(-1, y_train.shape[-1])  # (samples*days, 3)
    y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])     # (samples*days, 3)
    
    print(f"✓ Reshaped for scaling:")
    print(f"  y_train: {original_train_shape} -> {y_train_reshaped.shape}")
    print(f"  y_test: {original_test_shape} -> {y_test_reshaped.shape}")
    
    # Fit scaler on training data only
    target_scaler = StandardScaler()
    y_train_normalized = target_scaler.fit_transform(y_train_reshaped)
    y_test_normalized = target_scaler.transform(y_test_reshaped)
    
    # Reshape back to original format
    y_train_normalized = y_train_normalized.reshape(original_train_shape)
    y_test_normalized = y_test_normalized.reshape(original_test_shape)
    
    print(f"✓ Normalized target ranges:")
    print(f"  y_train: {y_train_normalized.min():.3f} - {y_train_normalized.max():.3f}")
    print(f"  y_test: {y_test_normalized.min():.3f} - {y_test_normalized.max():.3f}")
    print(f"  Mean: {y_train_normalized.mean():.3f}, Std: {y_train_normalized.std():.3f}")
    
    # Save scaler for later denormalization
    if save_scaler:
        joblib.dump(target_scaler, 'target_scaler.pkl')
        print(f"✅ Target scaler saved to 'target_scaler.pkl'")
    
    return y_train_normalized, y_test_normalized, target_scaler

def denormalize_predictions(predictions_normalized, target_scaler):
    """
    Convert normalized predictions back to actual dollar amounts
    """
    original_shape = predictions_normalized.shape
    
    # Reshape for denormalization
    pred_reshaped = predictions_normalized.reshape(-1, predictions_normalized.shape[-1])
    
    # Denormalize
    pred_actual = target_scaler.inverse_transform(pred_reshaped)
    
    # Reshape back
    pred_actual = pred_actual.reshape(original_shape)
    
    return pred_actual

print("✅ Target normalization functions defined")

✅ Target normalization functions defined


In [13]:
# Cell 4: Corrected - Handle data without Date column in features
def clean_and_prepare_data_fixed(df_transformed, target_data):
    """
    Clean dataframes when features don't have Date column
    """
    print("="*50)
    print("CLEANING AND PREPARING DATA FOR CNN-LSTM")
    print("="*50)
    
    # Step 1: Check original data
    print("Original data info:")
    print(f"df_transformed shape: {df_transformed.shape}")
    print(f"df_transformed columns: {list(df_transformed.columns)}")
    print(f"target_data shape: {target_data.shape}")
    print(f"target_data columns: {list(target_data.columns)}")
    
    # Check if data is already aligned by length
    if len(df_transformed) == len(target_data):
        print("✓ Data lengths match - assuming already aligned by row index")
        
        # Step 2: Pivot target data from long to wide format
        print("\n🔄 Pivoting target data to wide format...")
        
        # Add row index to help with pivoting
        target_with_index = target_data.copy()
        target_with_index['row_index'] = target_with_index.index
        
        # Create a day identifier (since we know there are 3 meal periods per day)
        target_with_index['day_id'] = target_with_index['row_index'] // 3
        
        target_pivot = target_with_index.pivot_table(
            index='day_id', 
            columns='MealPeriod', 
            values='CheckTotal', 
            fill_value=0
        ).reset_index()
        
        print(f"✓ Pivoted target shape: {target_pivot.shape}")
        print(f"✓ Pivoted target columns: {list(target_pivot.columns)}")
        
        # Step 3: Aggregate features to day level (average of 3 meal periods per day)
        print("\n📊 Aggregating features to day level...")
        
        # Add day_id to features
        df_features_with_day = df_transformed.copy()
        df_features_with_day['day_id'] = df_features_with_day.index // 3
        
        # Aggregate features by day (mean of the 3 meal periods)
        df_features_daily = df_features_with_day.groupby('day_id').mean().reset_index()
        df_features_daily = df_features_daily.drop('day_id', axis=1)
        
        print(f"✓ Aggregated features shape: {df_features_daily.shape}")
        
        # Step 4: Align the data
        target_values = target_pivot.drop('day_id', axis=1)
        
        # Ensure same number of rows
        min_rows = min(len(df_features_daily), len(target_values))
        df_features_final = df_features_daily.iloc[:min_rows]
        target_values_final = target_values.iloc[:min_rows]
        
        print(f"✓ Final aligned shapes:")
        print(f"Features: {df_features_final.shape}")
        print(f"Targets: {target_values_final.shape}")
        
    else:
        raise ValueError(f"Data length mismatch: features={len(df_transformed)}, targets={len(target_data)}")
    
    # Step 5: Clean data types and handle missing values
    print("\n🧹 Cleaning data types...")
    
    # Features: ensure all numeric
    df_features_clean = df_features_final.select_dtypes(include=[np.number])
    df_features_clean = df_features_clean.fillna(0).astype(np.float32)
    
    # Targets: ensure all numeric
    df_targets_clean = target_values_final.fillna(0).astype(np.float32)
    
    print(f"✅ Final cleaned data:")
    print(f"Features shape: {df_features_clean.shape}")
    print(f"Targets shape: {df_targets_clean.shape}")
    print(f"Target columns: {list(df_targets_clean.columns)}")
    print(f"Data lengths match: {len(df_features_clean) == len(df_targets_clean)}")
    
    return df_features_clean, df_targets_clean

def create_sequences_for_cnn_lstm_corrected(df_features, df_targets, sequence_length=30, forecast_horizon=7):
    """
    Create sequences from properly aligned and cleaned data
    """
    print("\n" + "="*50)
    print("CREATING SEQUENCES FOR CNN-LSTM")
    print("="*50)
    
    # Parameters
    SEQ_LENGTH = sequence_length
    FORECAST_HORIZON = forecast_horizon
    
    # Convert to arrays
    features = df_features.values
    targets = df_targets.values
    feature_columns = df_features.columns.tolist()
    target_columns = df_targets.columns.tolist()
    
    print(f"✓ Feature shape: {features.shape}")
    print(f"✓ Target shape: {targets.shape}")
    print(f"✓ Sequence length: {SEQ_LENGTH} days")
    print(f"✓ Forecast horizon: {FORECAST_HORIZON} days")
    print(f"✓ Target columns: {target_columns}")
    
    # Verify we have enough data
    min_data_needed = SEQ_LENGTH + FORECAST_HORIZON
    if len(features) < min_data_needed:
        raise ValueError(f"Not enough data. Need at least {min_data_needed} rows, got {len(features)}")
    
    # Create sequences
    X, y = [], []
    
    for i in range(SEQ_LENGTH, len(features) - FORECAST_HORIZON + 1):
        # Features: past SEQ_LENGTH days
        X.append(features[i-SEQ_LENGTH:i])
        
        # Targets: next FORECAST_HORIZON days
        y.append(targets[i:i+FORECAST_HORIZON])
    
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    print(f"✓ Final X shape: {X.shape}")  # (samples, sequence_length, features)
    print(f"✓ Final y shape: {y.shape}")  # (samples, forecast_horizon, revenue_streams)
    print(f"✓ X dtype: {X.dtype}")
    print(f"✓ y dtype: {y.dtype}")
    print(f"✓ Total sequences created: {len(X)}")
    
    # Show example of what each dimension means
    print(f"\n📊 Shape interpretation:")
    print(f"X: ({X.shape[0]} sequences, {X.shape[1]} days history, {X.shape[2]} features)")
    print(f"y: ({y.shape[0]} sequences, {y.shape[1]} days forecast, {y.shape[2]} revenue streams)")
    
    return X, y, feature_columns, target_columns

# Execute the corrected pipeline
try:
    # Step 1: Clean and prepare data without Date column dependency
    df_features_clean, df_targets_clean = clean_and_prepare_data_fixed(df_transformed, target_data)
    
    # Step 2: Create sequences
    X, y, feature_cols, target_cols = create_sequences_for_cnn_lstm_corrected(
        df_features_clean, df_targets_clean
    )
    
    print(f"\n🎉 SUCCESS! Sequences created successfully!")
    print(f"✓ Input sequences (X): {X.shape}")
    print(f"✓ Output sequences (y): {y.shape}")
    print(f"✓ Feature columns: {len(feature_cols)}")
    print(f"✓ Target columns: {target_cols}")
    print(f"✓ Data types: X={X.dtype}, y={y.dtype}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

CLEANING AND PREPARING DATA FOR CNN-LSTM
Original data info:
df_transformed shape: (1458, 65)
df_transformed columns: ['Year', 'CheckTotal', 'is_zero', 'IsRamadan', 'IsEid', 'IsPreRamadan', 'IsPostRamadan', 'IsLast10Ramadan', 'IsDSF', 'IsSummerEvent', 'IsNationalDay', 'IsNewYear', 'IsMarathon', 'IsGITEX', 'IsAirshow', 'IsFoodFestival', 'IsPreEvent', 'IsPostEvent', 'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'Meal_Breakfast', 'Meal_Dinner', 'Meal_Lunch', 'Event_Dubai-Airshow', 'Event_Dubai-Food-Festival', 'Event_Dubai-Marathon', 'Event_Dubai-Shopping-Festival', 'Event_Dubai-Summer-Surprises', 'Event_Eid-Adha', 'Event_Flag-Day', 'Event_GITEX-Technology-Week', 'Event_New-Year-Celebrations', 'Event_Normal', 'Event_Post-Dubai-Airshow', 'Event_Post-Dubai-Marathon', 'Event_Post-Eid-Adha', 'Event_Post-Flag-Day', 'Event_Post-GITEX-Technology-Week', 'Event_Post-New-Year-Celebrations', 'Event_Post-Ramadan-Recovery', 'Event_Post-Ramadan-Week1', 'Event_Post-Summer-Event', 'Event_Pre

In [14]:
print(X)
print("----------------------------------------------------------")
print(y)

[[[-0.5757663   0.6442341  -0.08310281 ... -0.3115533  -2.6150928
    4.957716  ]
  [-0.5757663   0.20041224 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.10293791 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  ...
  [-0.5757663   0.01185533 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.11598377 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663  -0.17047678 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]]

 [[-0.5757663   0.20041224 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.10293791 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.44010323 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  ...
  [-0.5757663   0.11598377 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663  -0.17047678 -0.08310281 ... -0.3115533   0.38239563
   -0.20170578]
  [-0.5757663   0.15700552 -0.08310281 ... -0.3115533   0.38239563
   -0.2

In [15]:
# Cell 5: Train-Test Split with clean data
print("="*30)
print("TRAIN-TEST SPLIT")
print("="*30)

# Time-based split (80% train, 20% test)
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"✓ Training sequences: {X_train.shape[0]}")
print(f"✓ Testing sequences: {X_test.shape[0]}")
print(f"✓ Input shape per sample: {X_train.shape[1:]}")
print(f"✓ Output shape per sample: {y_train.shape[1:]}")

# Verify data types
print(f"✓ X_train dtype: {X_train.dtype}")
print(f"✓ y_train dtype: {y_train.dtype}")
print(f"✓ X_test dtype: {X_test.dtype}")
print(f"✓ y_test dtype: {y_test.dtype}")

# Check for any problematic values
print(f"\n✓ Data quality check:")
print(f"X_train NaN count: {np.isnan(X_train).sum()}")
print(f"y_train NaN count: {np.isnan(y_train).sum()}")
print(f"X_train Inf count: {np.isinf(X_train).sum()}")
print(f"y_train Inf count: {np.isinf(y_train).sum()}")

print(f"\n✅ Data is ready for training!")

TRAIN-TEST SPLIT
✓ Training sequences: 360
✓ Testing sequences: 90
✓ Input shape per sample: (30, 65)
✓ Output shape per sample: (7, 3)
✓ X_train dtype: float32
✓ y_train dtype: float32
✓ X_test dtype: float32
✓ y_test dtype: float32

✓ Data quality check:
X_train NaN count: 0
y_train NaN count: 0
X_train Inf count: 0
y_train Inf count: 0

✅ Data is ready for training!


In [16]:
# Cell 6: Define CNN-LSTM model architecture
def build_cnn_lstm_model(input_shape, output_shape):
    """
    Build CNN-LSTM hybrid model for hotel revenue forecasting
    """
    print(f"✓ Building model with input shape: {input_shape}")
    print(f"✓ Output shape: {output_shape}")
    
    model = Sequential([
        # CNN layers for feature extraction
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape, name='conv1d_1'),
        Conv1D(filters=64, kernel_size=3, activation='relu', name='conv1d_2'),
        MaxPooling1D(pool_size=2, name='maxpool_1'),
        Dropout(0.2, name='dropout_1'),
        
        # More CNN layers
        Conv1D(filters=32, kernel_size=3, activation='relu', name='conv1d_3'),
        MaxPooling1D(pool_size=2, name='maxpool_2'),
        Dropout(0.2, name='dropout_2'),
        
        # LSTM layers for temporal patterns
        LSTM(100, return_sequences=True, name='lstm_1'),
        Dropout(0.3, name='dropout_3'),
        LSTM(50, return_sequences=False, name='lstm_2'),
        Dropout(0.3, name='dropout_4'),
        
        # Dense layers for final prediction
        Dense(100, activation='relu', name='dense_1'),
        Dropout(0.2, name='dropout_5'),
        Dense(np.prod(output_shape), activation='linear', name='dense_output'),
    ])
    
    # Reshape output to (forecast_days, revenue_streams)
    model.add(tf.keras.layers.Reshape(output_shape, name='reshape_output'))
    
    return model

print("✓ Model building function defined")

✓ Model building function defined


In [17]:
# Cell 6A: Apply Target Normalization
print("="*40)
print("APPLYING TARGET NORMALIZATION")
print("="*40)

# Store original targets for comparison
y_train_original = y_train.copy()
y_test_original = y_test.copy()

# Apply normalization
y_train_norm, y_test_norm, target_scaler = normalize_targets(y_train, y_test, save_scaler=True)

# Update variables for training
y_train = y_train_norm
y_test = y_test_norm

print(f"✅ Target normalization applied!")
print(f"✅ Training will use normalized targets")
print(f"✅ Original targets preserved for comparison")

# Show the difference
print(f"\n📊 Comparison:")
print(f"Original y_train range: ${y_train_original.min():.2f} - ${y_train_original.max():.2f}")
print(f"Normalized y_train range: {y_train.min():.3f} - {y_train.max():.3f}")

APPLYING TARGET NORMALIZATION
NORMALIZING TARGET VALUES
📊 Original target ranges:
  y_train: $5.00 - $10052.50
  y_test: $66.00 - $9657.00
✓ Reshaped for scaling:
  y_train: (360, 7, 3) -> (2520, 3)
  y_test: (90, 7, 3) -> (630, 3)
✓ Normalized target ranges:
  y_train: -1.588 - 9.795
  y_test: -1.334 - 9.494
  Mean: -0.000, Std: 1.000
✅ Target scaler saved to 'target_scaler.pkl'
✅ Target normalization applied!
✅ Training will use normalized targets
✅ Original targets preserved for comparison

📊 Comparison:
Original y_train range: $5.00 - $10052.50
Normalized y_train range: -1.588 - 9.795


In [18]:
# Cell 7: Build and compile the model
print("="*30)
print("BUILDING MODEL")
print("="*30)

# Define input and output shapes
input_shape = (X_train.shape[1], X_train.shape[2])  # (30, features)
output_shape = (y_train.shape[1], y_train.shape[2])  # (7, revenue_streams)

# Build model
model = build_cnn_lstm_model(input_shape, output_shape)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n" + "="*30)
print("MODEL ARCHITECTURE")
print("="*30)
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\n✓ Total parameters: {total_params:,}")

BUILDING MODEL
✓ Building model with input shape: (30, 65)
✓ Output shape: (7, 3)



MODEL ARCHITECTURE


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 28, 64)         │        12,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 26, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling1D)        │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 11, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling1D)        │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 100)         │        53,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_output (Dense)            │ (None, 21)             │         2,121 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_output (Reshape)        │ (None, 7, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,693 (475.36 KB)

 Trainable params: 121,693 (475.36 KB)

 Non-trainable params: 0 (0.00 B)


✓ Total parameters: 121,693


In [19]:
# Cell 8: Setup training callbacks
print("="*30)
print("TRAINING SETUP")
print("="*30)

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=15, 
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=5, 
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_cnn_lstm_model.h5', 
        save_best_only=True, 
        monitor='val_loss',
        verbose=1
    )
]

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100

print("✓ Callbacks configured:")
print("  - Early stopping (patience=15)")
print("  - Learning rate reduction (factor=0.5, patience=5)")
print("  - Model checkpoint (best_cnn_lstm_model.h5)")
print(f"✓ Batch size: {BATCH_SIZE}")
print(f"✓ Max epochs: {EPOCHS}")

TRAINING SETUP
✓ Callbacks configured:
  - Early stopping (patience=15)
  - Learning rate reduction (factor=0.5, patience=5)
  - Model checkpoint (best_cnn_lstm_model.h5)
✓ Batch size: 32
✓ Max epochs: 100


In [20]:
# Cell 9 Alternative: Comprehensive data cleaning and training
import tensorflow as tf

print("="*40)
print("COMPREHENSIVE DATA PREPARATION")
print("="*40)

def clean_and_prepare_data(X_train, y_train, X_test, y_test):
    """
    Comprehensive data cleaning for CNN-LSTM training
    """
    print("🔧 Cleaning and preparing data...")
    
    # Convert to numpy arrays if not already
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    # Check for object dtype issues
    if X_train.dtype == 'object':
        print("⚠️  X_train has object dtype - converting...")
        X_train = X_train.astype(np.float64)
    
    if y_train.dtype == 'object':
        print("⚠️  y_train has object dtype - converting...")
        y_train = y_train.astype(np.float64)
    
    if X_test.dtype == 'object':
        print("⚠️  X_test has object dtype - converting...")
        X_test = X_test.astype(np.float64)
    
    if y_test.dtype == 'object':
        print("⚠️  y_test has object dtype - converting...")
        y_test = y_test.astype(np.float64)
    
    # Handle NaN and infinite values
    print("🧹 Handling NaN and infinite values...")
    X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e6, neginf=-1e6)
    y_train = np.nan_to_num(y_train, nan=0.0, posinf=1e6, neginf=-1e6)
    X_test = np.nan_to_num(X_test, nan=0.0, posinf=1e6, neginf=-1e6)
    y_test = np.nan_to_num(y_test, nan=0.0, posinf=1e6, neginf=-1e6)
    
    # Convert to float32 (TensorFlow's preferred type)
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_test = y_test.astype(np.float32)
    
    # Final verification
    print(f"✓ Final data types:")
    print(f"  X_train: {X_train.dtype}, shape: {X_train.shape}")
    print(f"  y_train: {y_train.dtype}, shape: {y_train.shape}")
    print(f"  X_test: {X_test.dtype}, shape: {X_test.shape}")
    print(f"  y_test: {y_test.dtype}, shape: {y_test.shape}")
    
    # Check data ranges
    print(f"✓ Data ranges:")
    print(f"  X_train: [{X_train.min():.3f}, {X_train.max():.3f}]")
    print(f"  y_train: [{y_train.min():.3f}, {y_train.max():.3f}]")
    
    return X_train, y_train, X_test, y_test

# Clean the data
X_train_clean, y_train_clean, X_test_clean, y_test_clean = clean_and_prepare_data(
    X_train, y_train, X_test, y_test
)

print("\n" + "="*30)
print("STARTING TRAINING")
print("="*30)

# Train with cleaned data
try:
    history = model.fit(
        X_train_clean, y_train_clean,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test_clean, y_test_clean),
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n✅ Training completed successfully!")
    
    # Update variables for next cells
    X_train, y_train = X_train_clean, y_train_clean
    X_test, y_test = X_test_clean, y_test_clean
    
except Exception as e:
    print(f"❌ Training still failed: {e}")
    print("\n🔍 Additional debugging:")
    
    # More detailed debugging
    print(f"X_train unique dtypes: {set(str(x.dtype) for x in X_train.flatten()[:100])}")
    print(f"Sample X_train values: {X_train[0, 0, :10]}")
    print(f"Sample y_train values: {y_train[0, 0, :10]}")
    
    # Check if data contains any strings
    sample_x = X_train[0, 0, :]
    print(f"Sample X contains strings: {any(isinstance(x, str) for x in sample_x.flatten())}")

COMPREHENSIVE DATA PREPARATION
🔧 Cleaning and preparing data...
🧹 Handling NaN and infinite values...
✓ Final data types:
  X_train: float32, shape: (360, 30, 65)
  y_train: float32, shape: (360, 7, 3)
  X_test: float32, shape: (90, 30, 65)
  y_test: float32, shape: (90, 7, 3)
✓ Data ranges:
  X_train: [-2.615, 22.023]
  y_train: [-1.588, 9.795]

STARTING TRAINING
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9665 - mae: 0.6664 
Epoch 1: val_loss improved from inf to 3.43391, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - loss: 0.9689 - mae: 0.6665 - val_loss: 3.4339 - val_mae: 1.3141 - learning_rate: 0.0010
Epoch 2/100
 8/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7425 - mae: 0.6068 
Epoch 2: val_loss improved from 3.43391 to 3.04964, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.8072 - mae: 0.6184 - val_loss: 3.0496 - val_mae: 1.2144 - learning_rate: 0.0010
Epoch 3/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.8319 - mae: 0.6087
Epoch 3: val_loss improved from 3.04964 to 2.46417, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.8221 - mae: 0.6059 - val_loss: 2.4642 - val_mae: 1.0717 - learning_rate: 0.0010
Epoch 4/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6036 - mae: 0.5449
Epoch 4: val_loss improved from 2.46417 to 2.25386, saving model to best_cnn_lstm_model.h5


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.6289 - mae: 0.5532 - val_loss: 2.2539 - val_mae: 1.0263 - learning_rate: 0.0010
Epoch 5/100
10/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.8036 - mae: 0.6067
Epoch 5: val_loss did not improve from 2.25386
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.7809 - mae: 0.6014 - val_loss: 2.3177 - val_mae: 1.0313 - learning_rate: 0.0010
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6832 - mae: 0.5590
Epoch 6: val_loss did not improve from 2.25386
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.6840 - mae: 0.5599 - val_loss: 2.2775 - val_mae: 1.0194 - learning_rate: 0.0010
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7059 - mae: 0.5670 
Epoch 7: val_loss did not improve from 2.25386
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7049 - mae: 0.5669 - val_loss: 2.7672 - val_mae: 1.1201 - learning_rate: 0.0010
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7838 - mae: 0.5843
Epoch 8: val_lo

In [21]:
# Cell 10: Model Evaluation with Denormalization
print("="*30)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*30)

# IMPORTANT: Model was trained on NORMALIZED targets
# We need to denormalize predictions for evaluation
print("📊 NOTE: Model trained on normalized targets")
print("📊 Denormalizing predictions to actual dollar amounts")

# Make predictions on normalized test set
y_pred_normalized = model.predict(X_test, verbose=0)

# Load scaler and denormalize predictions
try:
    target_scaler = joblib.load('target_scaler.pkl')
    y_pred_actual = denormalize_predictions(y_pred_normalized, target_scaler)
    y_test_actual = y_test_original  # Use original non-normalized test targets
    
    print(f"✅ Predictions denormalized successfully")
    
except Exception as e:
    print(f"⚠️  Could not load scaler: {e}")
    print(f"📊 Using normalized predictions for evaluation")
    y_pred_actual = y_pred_normalized
    y_test_actual = y_test

# Debug shapes
print(f"\n🔍 Shape Debugging:")
print(f"X_test shape: {X_test.shape}")
print(f"y_test_actual shape: {y_test_actual.shape}")
print(f"y_pred_actual shape: {y_pred_actual.shape}")

# Define revenue stream names
revenue_streams = ['Breakfast', 'Dinner', 'Lunch']
print(f"\n📊 Revenue streams: {revenue_streams}")

# Show data ranges (should be in dollars after denormalization)
print(f"\n💰 Denormalized Value Ranges:")
print(f"  Actual revenue: ${y_test_actual.min():.2f} - ${y_test_actual.max():.2f}")
print(f"  Predicted revenue: ${y_pred_actual.min():.2f} - ${y_pred_actual.max():.2f}")

# Calculate metrics on actual dollar amounts
y_test_flat = y_test_actual.reshape(-1)
y_pred_flat = y_pred_actual.reshape(-1)

mae = mean_absolute_error(y_test_flat, y_pred_flat)
mse = mean_squared_error(y_test_flat, y_pred_flat)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_flat - y_pred_flat) / (np.abs(y_test_flat) + 1e-8))) * 100

print(f"\n✅ Overall Test Metrics (in USD):")
print(f"  MAE: ${mae:.2f}")
print(f"  RMSE: ${rmse:.2f}")
print(f"  MAPE: {mape:.2f}%")

# Performance by revenue stream
print(f"\n✅ Performance by Revenue Stream:")
for stream_idx, stream_name in enumerate(revenue_streams):
    stream_mae = mean_absolute_error(
        y_test_actual[:, :, stream_idx].reshape(-1), 
        y_pred_actual[:, :, stream_idx].reshape(-1)
    )
    stream_corr = np.corrcoef(
        y_test_actual[:, :, stream_idx].reshape(-1),
        y_pred_actual[:, :, stream_idx].reshape(-1)
    )[0, 1]
    print(f"  {stream_name}: MAE = ${stream_mae:.2f}, Correlation = {stream_corr:.3f}")

# Sample predictions
print(f"\n✅ Sample Predictions (First sequence - in USD):")
print("Day | Breakfast_Actual | Breakfast_Pred | Dinner_Actual | Dinner_Pred | Lunch_Actual | Lunch_Pred")
print("-" * 95)
for day in range(min(7, y_test_actual.shape[1])):
    print(f"{day+1:2d}  | ${y_test_actual[0, day, 0]:11.2f}     | ${y_pred_actual[0, day, 0]:9.2f}     | "
          f"${y_test_actual[0, day, 1]:8.2f}     | ${y_pred_actual[0, day, 1]:6.2f}     | "
          f"${y_test_actual[0, day, 2]:7.2f}     | ${y_pred_actual[0, day, 2]:5.2f}")

print(f"\n✅ Evaluation complete with denormalized predictions!")

COMPREHENSIVE MODEL EVALUATION
📊 NOTE: Model trained on normalized targets
📊 Denormalizing predictions to actual dollar amounts
✅ Predictions denormalized successfully

🔍 Shape Debugging:
X_test shape: (90, 30, 65)
y_test_actual shape: (90, 7, 3)
y_pred_actual shape: (90, 7, 3)

📊 Revenue streams: ['Breakfast', 'Dinner', 'Lunch']

💰 Denormalized Value Ranges:
  Actual revenue: $66.00 - $9657.00
  Predicted revenue: $481.21 - $5506.60

✅ Overall Test Metrics (in USD):
  MAE: $870.10
  RMSE: $1273.38
  MAPE: 75.66%

✅ Performance by Revenue Stream:
  Breakfast: MAE = $776.84, Correlation = 0.484
  Dinner: MAE = $1218.92, Correlation = 0.678
  Lunch: MAE = $614.53, Correlation = 0.546

✅ Sample Predictions (First sequence - in USD):
Day | Breakfast_Actual | Breakfast_Pred | Dinner_Actual | Dinner_Pred | Lunch_Actual | Lunch_Pred
-----------------------------------------------------------------------------------------------
 1  | $    2466.00     | $  1767.07     | $ 6548.00     | $5111.87

In [23]:
# FEATURE RELEVANCE ANALYSIS - Fixed version for your CNN-LSTM notebook
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("FEATURE RELEVANCE ANALYSIS FOR CNN-LSTM MODEL")
print("="*80)

# Debug the data shapes first
print(f"Analyzing data shapes:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Number of features: {len(feature_cols)}")

# Check if we're using normalized targets
if 'y_train_norm' in locals():
    print("Using normalized targets for analysis")
    y_analysis = y_train_norm
else:
    print("Using original targets for analysis")
    y_analysis = y_train

print(f"Analysis target shape: {y_analysis.shape}")

# Flatten the data for analysis - CORRECTED VERSION
print("\nFlattening sequences for feature analysis...")

# Use only training data for consistency
X_flat = X_train.reshape(-1, X_train.shape[-1])  # (train_sequences*timesteps, features)
y_flat = y_analysis.reshape(-1, y_analysis.shape[-1])  # (train_sequences*timesteps, targets)

print(f"Flattened shapes: X_flat={X_flat.shape}, y_flat={y_flat.shape}")

# Verify shapes match
if X_flat.shape[0] != y_flat.shape[0]:
    print(f"ERROR: Shape mismatch detected!")
    print(f"X_flat samples: {X_flat.shape[0]}")
    print(f"y_flat samples: {y_flat.shape[0]}")
    
    # Fix by using the minimum length
    min_samples = min(X_flat.shape[0], y_flat.shape[0])
    X_flat = X_flat[:min_samples]
    y_flat = y_flat[:min_samples]
    print(f"Fixed shapes: X_flat={X_flat.shape}, y_flat={y_flat.shape}")

target_names = ['Breakfast', 'Dinner', 'Lunch']

# 1. RANDOM FOREST FEATURE IMPORTANCE
print("\n" + "="*60)
print("1. RANDOM FOREST FEATURE IMPORTANCE")
print("="*60)

rf_results = {}
for i, target_name in enumerate(target_names):
    print(f"\nAnalyzing {target_name} revenue...")
    
    try:
        rf = RandomForestRegressor(
            n_estimators=50,  # Reduced for speed
            random_state=42, 
            max_depth=8,
            min_samples_split=20,
            n_jobs=-1
        )
        rf.fit(X_flat, y_flat[:, i])
        
        rf_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': rf.feature_importances_
        }).sort_values('importance', ascending=False)
        
        rf_results[target_name] = rf_importance
        
        print(f"Top 10 features for {target_name}:")
        for idx, row in rf_importance.head(10).iterrows():
            print(f"  {idx+1:2d}. {row['feature']:<35} {row['importance']:.4f}")
    
    except Exception as e:
        print(f"Error in Random Forest for {target_name}: {e}")
        continue

# 2. CORRELATION ANALYSIS
print("\n" + "="*60)
print("2. CORRELATION ANALYSIS")
print("="*60)

corr_results = {}
for i, target_name in enumerate(target_names):
    print(f"\nCalculating correlations for {target_name}...")
    
    try:
        correlations = []
        for j in range(len(feature_cols)):
            # Handle any NaN values
            feature_vals = X_flat[:, j]
            target_vals = y_flat[:, i]
            
            # Remove NaN pairs
            mask = ~(np.isnan(feature_vals) | np.isnan(target_vals))
            if mask.sum() > 10:  # Need at least 10 valid pairs
                corr, _ = pearsonr(feature_vals[mask], target_vals[mask])
                correlations.append(abs(corr) if not np.isnan(corr) else 0)
            else:
                correlations.append(0)
        
        corr_importance = pd.DataFrame({
            'feature': feature_cols,
            'abs_correlation': correlations
        }).sort_values('abs_correlation', ascending=False)
        
        corr_results[target_name] = corr_importance
        
        print(f"Top 10 correlations for {target_name}:")
        for idx, row in corr_importance.head(10).iterrows():
            print(f"  {idx+1:2d}. {row['feature']:<35} {row['abs_correlation']:.4f}")
    
    except Exception as e:
        print(f"Error in correlation analysis for {target_name}: {e}")
        continue

# 3. SIMPLE VARIANCE ANALYSIS (Alternative to LASSO)
print("\n" + "="*60)
print("3. FEATURE VARIANCE ANALYSIS")
print("="*60)

# Calculate feature variance (low variance = less informative)
feature_variances = np.var(X_flat, axis=0)
variance_df = pd.DataFrame({
    'feature': feature_cols,
    'variance': feature_variances
}).sort_values('variance', ascending=False)

print("Top 10 features by variance:")
for idx, row in variance_df.head(10).iterrows():
    print(f"  {idx+1:2d}. {row['feature']:<35} {row['variance']:.4f}")

print("\nBottom 10 features by variance (potentially redundant):")
for idx, row in variance_df.tail(10).iterrows():
    print(f"  {idx+1:2d}. {row['feature']:<35} {row['variance']:.4f}")

# 4. COMBINED RANKING ANALYSIS
print("\n" + "="*60)
print("4. COMBINED FEATURE RANKING")
print("="*60)

combined_rankings = {}

for target_name in target_names:
    if target_name not in rf_results or target_name not in corr_results:
        continue
        
    print(f"\nCombined analysis for {target_name}:")
    
    # Get rankings from each method
    rf_rank = rf_results[target_name].reset_index(drop=True)
    rf_rank['rf_rank'] = rf_rank.index + 1
    
    corr_rank = corr_results[target_name].reset_index(drop=True)
    corr_rank['corr_rank'] = corr_rank.index + 1
    
    # Add variance ranking
    var_rank = variance_df.reset_index(drop=True)
    var_rank['var_rank'] = var_rank.index + 1
    
    # Merge all rankings
    combined = rf_rank[['feature', 'rf_rank', 'importance']].merge(
        corr_rank[['feature', 'corr_rank', 'abs_correlation']], on='feature'
    ).merge(
        var_rank[['feature', 'var_rank', 'variance']], on='feature'
    )
    
    # Calculate average rank (lower is better)
    combined['avg_rank'] = combined[['rf_rank', 'corr_rank', 'var_rank']].mean(axis=1)
    combined = combined.sort_values('avg_rank')
    
    combined_rankings[target_name] = combined
    
    print("Top 15 features by combined ranking:")
    for idx, row in combined.head(15).iterrows():
        print(f"  {idx+1:2d}. {row['feature']:<35} Rank: {row['avg_rank']:.1f}")

# 5. FEATURE CATEGORY ANALYSIS
print("\n" + "="*60)
print("5. FEATURE CATEGORY ANALYSIS")
print("="*60)

def categorize_feature(feature_name):
    if feature_name.startswith('Meal_'):
        return 'Meal_Period'
    elif feature_name.startswith('Event_'):
        return 'Events'
    elif feature_name.startswith('Tourism_'):
        return 'Tourism'
    elif feature_name.startswith('Impact_'):
        return 'Revenue_Impact'
    elif feature_name.endswith(('_sin', '_cos')):
        return 'Cyclical'
    elif feature_name.startswith('Is'):
        return 'Event_Flags'
    else:
        return 'Core'

# Analyze feature categories
all_features_df = pd.DataFrame({'feature': feature_cols})
all_features_df['category'] = all_features_df['feature'].apply(categorize_feature)

category_counts = all_features_df['category'].value_counts()
print("Feature count by category:")
for category, count in category_counts.items():
    print(f"  {category:<15}: {count:2d} features")

# 6. OVERALL FEATURE RECOMMENDATIONS
print("\n" + "="*60)
print("6. FEATURE REDUCTION RECOMMENDATIONS")
print("="*60)

# Find top features across all targets
all_top_features = set()
top_counts = Counter()

for target_name in target_names:
    if target_name in combined_rankings:
        top_20 = combined_rankings[target_name].head(20)['feature'].tolist()
        all_top_features.update(top_20)
        for feature in top_20:
            top_counts[feature] += 1

# Sort by how many targets find this feature important
recommended_features = sorted(all_top_features, key=lambda x: top_counts[x], reverse=True)

print(f"RECOMMENDED FEATURE SET ({len(recommended_features)} features):")
print("Features ranked in top 20 for at least one revenue stream:")

for i, feature in enumerate(recommended_features[:25], 1):  # Show top 25
    count = top_counts[feature]
    stars = "★" * count
    print(f"  {i:2d}. {feature:<35} {stars} ({count}/3)")

# 7. FEATURES TO REMOVE
print("\n" + "="*60)
print("7. FEATURES LIKELY TO REMOVE")
print("="*60)

# Features with consistently low rankings
bottom_features = []
for target_name in target_names:
    if target_name in combined_rankings:
        bottom_15 = combined_rankings[target_name].tail(15)['feature'].tolist()
        bottom_features.extend(bottom_15)

# Features that appear in bottom rankings multiple times
bottom_counts = Counter(bottom_features)
likely_redundant = [feature for feature, count in bottom_counts.items() if count >= 2]

print("Features consistently ranked low:")
for feature in sorted(likely_redundant)[:15]:  # Show top 15 candidates for removal
    count = bottom_counts[feature]
    print(f"  - {feature:<35} (bottom 15 for {count}/3 targets)")

# 8. SUMMARY AND RECOMMENDATIONS
print("\n" + "="*60)
print("8. SUMMARY AND RECOMMENDATIONS")
print("="*60)

print(f"Current model:")
print(f"  Total features: {len(feature_cols)}")
print(f"  Training sequences: {X_train.shape[0]}")
print(f"  Samples per feature: {X_flat.shape[0] / len(feature_cols):.1f}")

optimal_features = min(30, len(recommended_features))
print(f"\nRecommended optimization:")
print(f"  Keep top features: {optimal_features}")
print(f"  Remove features: {len(feature_cols) - optimal_features}")
print(f"  Reduction: {(len(feature_cols) - optimal_features) / len(feature_cols) * 100:.1f}%")
print(f"  New samples per feature: {X_flat.shape[0] / optimal_features:.1f}")

print(f"\nTOP {optimal_features} FEATURES TO KEEP:")
for i, feature in enumerate(recommended_features[:optimal_features], 1):
    print(f"  {i:2d}. {feature}")

# 9. CREATE FEATURE SELECTION CODE
print("\n" + "="*60)
print("9. CODE TO IMPLEMENT FEATURE REDUCTION")
print("="*60)

top_features_list = recommended_features[:optimal_features]
print("# Copy this code to implement feature reduction:")
print(f"recommended_features = {top_features_list}")
print("\n# Get indices of recommended features")
print("feature_indices = [feature_cols.index(f) for f in recommended_features if f in feature_cols]")
print(f"print(f'Found {{len(feature_indices)}} feature indices')")
print("\n# Reduce your dataset")
print("X_train_reduced = X_train[:, :, feature_indices]")
print("X_test_reduced = X_test[:, :, feature_indices]")
print("feature_cols_reduced = [feature_cols[i] for i in feature_indices]")
print(f"print(f'Reduced from {{X_train.shape}} to {{X_train_reduced.shape}}')")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("Use the recommended_features list above to reduce your feature set.")
print("="*80)

FEATURE RELEVANCE ANALYSIS FOR CNN-LSTM MODEL
Analyzing data shapes:
X shape: (450, 30, 65)
y shape: (450, 7, 3)
Number of features: 65
Using normalized targets for analysis
Analysis target shape: (360, 7, 3)

Flattening sequences for feature analysis...
Flattened shapes: X_flat=(10800, 65), y_flat=(2520, 3)
ERROR: Shape mismatch detected!
X_flat samples: 10800
y_flat samples: 2520
Fixed shapes: X_flat=(2520, 65), y_flat=(2520, 3)

1. RANDOM FOREST FEATURE IMPORTANCE

Analyzing Breakfast revenue...
Top 10 features for Breakfast:
  20. Month_cos                           0.4697
   2. CheckTotal                          0.1208
  59. Tourism_0                           0.1020
  21. DayOfWeek_sin                       0.0752
  22. DayOfWeek_cos                       0.0382
  63. Impact_-1                           0.0325
  56. Event_Ramadan-First10Days           0.0312
   8. IsLast10Ramadan                     0.0280
  57. Event_Ramadan-Last10Days            0.0253
  64. Impact_0          